# PRÁCTICA B2-2 #

## MÓDULO DE GESTIÓN DE RIESGOS ##
### Escenarios de Estrés y Cambios de Régimen de Mercado ###

### Datos básicos: ###
- Práctica en grupos de dos personas
- Entrega el día 15 de febrero a través del aula virtual.
- Los entregables son un notebook de Python y un resumen ejecutivo en formato PDF.

### Objetivo de la práctica ###
El objetivo de esta práctica es rediseñar un motor de stress testing en Python capaz de
capturar el riesgo de cola y los cambios de régimen, identificar cuándo el mercado entra en
“crisis” y cuantificar el riesgo real cuando la diversificación desaparece. El motor de
simulación deberá utilizarse explícitamente para construir Escenarios de Estrés cuyo
objetivo sea “romper la cartera”, forzando condiciones adversas y económicamente
coherentes, y cuantificando pérdidas extremas mediante VaR del 99% y Expected Shortfall
(CVaR).

### Fase 0 - Preparacion y Estructura del Proyecto ###

### Librerias ###

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yfinance as yf
from datetime import date
from pathlib import Path
from typing import List
import pandas_datareader as pdr
from dataclasses import dataclass, field
from typing import Dict, List

### Variables ###

In [61]:
seed = 42

BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_BRONZE_DIR = DATA_DIR / "bronze"
DATA_SILVER_DIR = DATA_DIR / "silver"
DATA_GOLD_DIR = DATA_DIR / "gold"
FIGURES_DIR = BASE_DIR / "figures"
START_DATE = "2006-01-01"
END_DATE = date.today().isoformat()


### Funciones ###

In [ ]:

def fetch_equities(tickers: List[str], start: str, end: str) -> pd.DataFrame:
    """Fetch adjusted close prices for a list of tickers using yfinance.

    Returns a DataFrame with dates as index and tickers as columns.
    """

    equities = yf.download(tickers, start=start, end=end, progress=False, threads=True, auto_adjust=True)['Close']
    
    tickers_join = tickers
    
    if isinstance(tickers, list):
        tickers_join = '_'.join(tickers)
    
    equities_path = DATA_BRONZE_DIR / f"equities_adj_close_{tickers_join}.csv"
    equities.to_csv(equities_path)
   
    return equities.sort_index()


def fetch_us_yields(tickers, start: str, end: str) -> pd.DataFrame:
    """Fetch US 10-year and 2-year constant maturity yields from FRED.

    The FRED series are `GS10` and `GS2` (daily; many days are reported).
    """

    yields = pdr.DataReader(tickers, "fred", start, end)
    yields.index = pd.to_datetime(yields.index)
    
    tickers_join = tickers

    if isinstance(tickers, list):
        tickers_join = '_'.join(tickers)
    
    yields_path = DATA_BRONZE_DIR / f"us_yields_{tickers_join}.csv"
    yields.to_csv(yields_path)

    return yields.sort_index()


def combine_and_fill(equities: pd.DataFrame, yields: pd.DataFrame) -> pd.DataFrame:
    """Combine equities and yields into a single DataFrame and forward-fill missing data.

    Forward-filling is a pragmatic choice for aligning daily series with different reporting
    calendars; students can choose alternative imputation methods later.
    """

    combined = pd.concat([equities, yields], axis=1)
    combined = combined.sort_index()
    
    # Forward-fill in US10Y and US2Y columns only, to avoid filling equity data with yield values
    combined[['GS10','GS2']] = combined[['GS10','GS2']].ffill()
    
    combined_path = DATA_SILVER_DIR / "market_data_combined.csv"
    combined.to_csv(combined_path)

    return combined
      
def validate_data_quality(df: pd.DataFrame) -> None:
    """Perform basic data quality checks and print summary statistics."""
    print("Data Quality Report:")
    print(f"Number of rows: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print("\nMissing Values:")
    print(df.isna().sum())
    print("\nSummary Statistics:")
    print(df.describe())
    
    
def yield_curve_slope(y10: pd.Series, y2: pd.Series) -> pd.Series:
    return (y10 - y2)

### Clases ###

In [63]:
@dataclass
class Portfolio:
    assets: Dict[str, str]  # {"AAPL": "equity", "IEF": "yield"}
    
    prices: pd.DataFrame = field(init=False)
    returns: pd.DataFrame = field(init=False)
    weights: pd.DataFrame = field(init=False)
    portfolio_returns: pd.Series = field(init=False)

    def __post_init__(self):
        self._load_prices()
        self._compute_returns()
        self._compute_dynamic_weights()
        self._compute_portfolio_returns()
        
    def _load_prices(self):
        equities = [k for k, v in self.assets.items() if v == "equity"]
        yields = [k for k, v in self.assets.items() if v == "yield"]

        df_equities = pd.DataFrame()
        df_yields = pd.DataFrame()

        if equities:
            df_equities = fetch_equities(equities, start=START_DATE, end=END_DATE)

        if yields:
            df_yields = fetch_us_yields(yields,start=START_DATE, end=END_DATE)

        self.prices = combine_and_fill(df_equities, df_yields)

    def _compute_returns(self):
        self.returns = self.prices.pct_change()

    def _compute_dynamic_weights(self):
        asset_exists = ~self.prices.isna()

        n_assets = asset_exists.sum(axis=1)

        self.weights = asset_exists.div(n_assets, axis=0)

        self.weights = self.weights.fillna(0.0)

    def _compute_portfolio_returns(self):
        self.portfolio_returns = (self.returns * self.weights).sum(axis=1)

    def cumulative_return(self) -> pd.Series:
        return (1 + self.portfolio_returns).cumprod()

    def drawdown(self) -> pd.Series:
        wealth = self.cumulative_return()
        peak = wealth.cummax()
        return (wealth - peak) / peak

    def max_drawdown(self) -> float:
        return self.drawdown().min()

    def volatility(self, annualized: bool = True) -> float:
        vol = self.portfolio_returns.std()
        return vol * np.sqrt(252) if annualized else vol

    def mean_return(self, annualized: bool = True) -> float:
        mu = self.portfolio_returns.mean()
        return mu * 252 if annualized else mu

    def sharpe_ratio(self) -> float:
        return self.mean_return() / self.volatility()

    def var_cvar(self, alpha: float = 0.99):
        var = self.portfolio_returns.quantile(1 - alpha)
        cvar = self.portfolio_returns[self.portfolio_returns <= var].mean()
        return var, cvar

    def summary(self) -> pd.Series:
        var_99, cvar_99 = self.var_cvar(0.99)

        return pd.Series({
            "Mean Return (ann)": self.mean_return(),
            "Volatility (ann)": self.volatility(),
            "Sharpe": self.sharpe_ratio(),
            "Max Drawdown": self.max_drawdown(),
            "VaR 99%": var_99,
            "CVaR 99%": cvar_99
        })
        
    def portfolio_composition_table(self) -> pd.DataFrame:
        weights_pct = self.weights * 100

        asset_values = self.prices * self.weights

        data = {}

        for asset in self.prices.columns:
            data[(asset, "weight_%")] = weights_pct[asset]
            data[(asset, "price")] = self.prices[asset]
            data[(asset, "value")] = asset_values[asset]

        df = pd.DataFrame(data)

        df.columns = pd.MultiIndex.from_tuples(df.columns)

        df["portfolio_value"] = asset_values.sum(axis=1)

        df["portfolio_return_%"] = df["portfolio_value"].pct_change() * 100

        portfolio_table_path = DATA_GOLD_DIR / "portfolio_composition.csv"
        df.to_csv(portfolio_table_path)

        return df
    
    def plot_portfolio(self):
        cumulative_returns = self.cumulative_return()
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=cumulative_returns)
        plt.title("Cumulative Return of the Portfolio")
        plt.xlabel("Date")
        plt.ylabel("Cumulative Return")
        plt.tight_layout()
        plt.show()
        
        chart_path = FIGURES_DIR / f"portfolio_returns_chart.png"
        plt.savefig(chart_path)
        plt.close()
        
    def plot_chart_per_asset(self):
        for col in self.prices.columns:
            plt.figure(figsize=(12, 6))
            sns.lineplot(data=self.prices[col])
            plt.title(f"{col} Price Over Time")
            plt.xlabel("Date")
            plt.ylabel("Price")
            plt.tight_layout()
            chart_path = FIGURES_DIR / f"{col}_price_chart.png"
            plt.savefig(chart_path)
            plt.close()


### Datos ###

In [64]:
def portfolio() -> None:
    """Main entry point for data fetching.

    The multi-asset portfolio (equal-weight, no rebalancing) includes the following
    equity/ETF tickers and fixed-income items:

    AAPL, AMZN, BAC, BRK-B, CVX, ENPH, GLD, GME, GOOGL, JNJ, JPM, MSFT, NVDA, PG, XOM,
    HYG (corporate high-yield ETF), plus US 10y and 2y yields from FRED.
    """

    assets = {
        "AAPL": "equity",
        "AMZN": "equity",
        "BAC": "equity",
        "BRK-B": "equity",
        "CVX": "equity",
        "ENPH": "equity",
        "GLD": "equity",
        "GME": "equity",
        "GOOGL": "equity",
        "JNJ": "equity",
        "JPM": "equity",
        "MSFT": "equity",
        "NVDA": "equity",
        "PG": "equity",
        "XOM": "equity",
        "HYG": "equity",
        "GS10": "yield",
        "GS2": "yield"
    }

    portfolio = Portfolio(assets)

    return portfolio
    
def market_risk() -> None:
    # Mercado
    sp500 = fetch_equities(tickers=["^GSPC"], start=START_DATE, end=END_DATE)
    sp500_ret = sp500.pct_change()    
    
    vix = fetch_equities(tickers=["^VIX"], start=START_DATE, end=END_DATE)
    vix_ret = vix.pct_change()

    # Juros
    y10 = fetch_us_yields("GS10", start=START_DATE, end=END_DATE)
    y2 = fetch_us_yields("GS2", start=START_DATE, end=END_DATE)
    
    y10_chg = y10.pct_change()
    y2_chg = y2.pct_change()
    slope = yield_curve_slope(y10['GS10'], y2['GS2']).rename("yield_slope")

    # Crédito
    hy_spread = fetch_us_yields("BAMLH0A0HYM2", start=START_DATE, end=END_DATE)
    hy_spread_chg = hy_spread.pct_change()

    # Combinação
    df = pd.concat(
        [
            sp500_ret,
            vix_ret,
            y10_chg,
            y2_chg,
            slope,
            hy_spread_chg
        ],
        axis=1
    )
    
    df[['GS10','GS2','yield_slope']] = df[['GS10','GS2','yield_slope']].ffill()
    
    df_path = DATA_SILVER_DIR / "market_risk_factors.csv"
    df.to_csv(df_path)
    
    return df

if __name__ == "__main__":
    portfolio = portfolio()
    risk_market = market_risk()
    print(risk_market.head())

['GS10', 'GS2'] <class 'list'>


C:\Users\piett\AppData\Local\Temp\ipykernel_25012\2878745184.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.returns = self.prices.pct_change()


GS10 <class 'str'>
GS2 <class 'str'>
BAMLH0A0HYM2 <class 'str'>
               ^GSPC      ^VIX  GS10  GS2  yield_slope  BAMLH0A0HYM2
2006-01-01       NaN       NaN   NaN  NaN         0.02           NaN
2006-01-02       NaN       NaN   NaN  NaN         0.02           NaN
2006-01-03       NaN       NaN   NaN  NaN         0.02           NaN
2006-01-04  0.003673  0.020646   NaN  NaN         0.02     -0.010724
2006-01-05  0.000016 -0.005277   NaN  NaN         0.02     -0.013550


C:\Users\piett\AppData\Local\Temp\ipykernel_25012\488034089.py:54: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  hy_spread_chg = hy_spread.pct_change()
